<a href="https://colab.research.google.com/github/NicoPerotto/challenge2-data-science-LATAM/blob/main/TelecomX_LATAM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Importación y procedimientos

In [192]:
import pandas as pd
import numpy as np

##Procedimiento para mejorar la visualziacion de datos

In [193]:
def info(df):
    print(f"{'#':<3} {'Columna':<20} {'Tipo':<12} {'Indet.':<10} {'Valores Únicos / Ejemplo'}")
    print("-" * 90)

    for i, col in enumerate(df.columns):
        dtype = str(df[col].dtype)

        # 1. Conteo de vacíos (usamos apply para evitar errores de tipo)
        # Esto cuenta nulos reales, espacios y strings vacíos de forma segura
        vacios = df[col].apply(lambda x: x in ['', ' ', np.nan, None]).sum()

        try:
            # 2. Intentamos sacar valores únicos
            valores = df[col].unique().tolist()

            if len(valores) < 10:
                resumen = f"{valores}"
            else:
                resumen = f"{len(valores)} valores diferentes"
        except TypeError:
            # 3. Si falla por tener diccionarios/listas, damos un resumen alternativo
            resumen = "[Contiene diccionarios o listas]"

        print(f"{i:<3} {col:<20} {dtype:<12} {vacios:<10} {resumen}")

    print("-" * 90)
    filas, columnas = df.shape
    memoria = df.memory_usage(deep=True).sum() / (1024 ** 2) # Convertir a MB

    print(f"Tamaño total: {filas} filas y {columnas} columnas")

#📌 Extracción

In [194]:
dataFrame = pd.read_json('https://raw.githubusercontent.com/ingridcristh/challenge2-data-science-LATAM/refs/heads/main/TelecomX_Data.json')
dataFrame.head()

,customerID,Churn,customer,phone,internet,account
0,0002-ORFBO,No,"{'gender': 'Female', 'SeniorCitizen': 0, 'Part...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'DSL', 'OnlineSecurity': '...","{'Contract': 'One year', 'PaperlessBilling': '..."
1,0003-MKNFE,No,"{'gender': 'Male', 'SeniorCitizen': 0, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'Yes'}","{'InternetService': 'DSL', 'OnlineSecurity': '...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
2,0004-TLHLJ,Yes,"{'gender': 'Male', 'SeniorCitizen': 0, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
3,0011-IGKFF,Yes,"{'gender': 'Male', 'SeniorCitizen': 1, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
4,0013-EXCHZ,Yes,"{'gender': 'Female', 'SeniorCitizen': 1, 'Part...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."


#🔧 Transformación

 ## Comprender la estructura del dataset y el significado de sus columnas.

In [195]:
info(dataFrame)

#   Columna              Tipo         Indet.     Valores Únicos / Ejemplo
------------------------------------------------------------------------------------------
0   customerID           object       0          7267 valores diferentes
1   Churn                object       224        ['No', 'Yes', '']
2   customer             object       0          [Contiene diccionarios o listas]
3   phone                object       0          [Contiene diccionarios o listas]
4   internet             object       0          [Contiene diccionarios o listas]
5   account              object       0          [Contiene diccionarios o listas]
------------------------------------------------------------------------------------------
Tamaño total: 7267 filas y 6 columnas


In [196]:
columnas_a_normalizar = list(dataFrame.columns)
columnas_a_normalizar.remove('customerID')
columnas_a_normalizar.remove('Churn')

for col in columnas_a_normalizar:
    # Creamos el dataframe temporal normalizado
    temp_df = pd.json_normalize(dataFrame[col])
    dataFrame = pd.concat([dataFrame.drop(columns=[col]), temp_df], axis=1)
dataFrame.set_index('customerID', inplace=True)
dataFrame.head()

,Churn,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,Charges.Monthly,Charges.Total
customerID,,,,,,,,,,,,,,,,,,,,
0002-ORFBO,No,Female,0,Yes,Yes,9,Yes,No,DSL,No,Yes,No,Yes,Yes,No,One year,Yes,Mailed check,65.6,593.3
0003-MKNFE,No,Male,0,No,No,9,Yes,Yes,DSL,No,No,No,No,No,Yes,Month-to-month,No,Mailed check,59.9,542.4
0004-TLHLJ,Yes,Male,0,No,No,4,Yes,No,Fiber optic,No,No,Yes,No,No,No,Month-to-month,Yes,Electronic check,73.9,280.85
0011-IGKFF,Yes,Male,1,Yes,No,13,Yes,No,Fiber optic,No,Yes,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,98.0,1237.85
0013-EXCHZ,Yes,Female,1,Yes,No,3,Yes,No,Fiber optic,No,No,No,Yes,Yes,No,Month-to-month,Yes,Mailed check,83.9,267.4


## Verificar si hay problemas en los datos que puedan afectar el análisis.

In [197]:
dataFrame = dataFrame.convert_dtypes()
info(dataFrame)

#   Columna              Tipo         Indet.     Valores Únicos / Ejemplo
------------------------------------------------------------------------------------------
0   Churn                string       224        ['No', 'Yes', '']
1   gender               string       0          ['Female', 'Male']
2   SeniorCitizen        Int64        0          [0, 1]
3   Partner              string       0          ['Yes', 'No']
4   Dependents           string       0          ['Yes', 'No']
5   tenure               Int64        0          73 valores diferentes
6   PhoneService         string       0          ['Yes', 'No']
7   MultipleLines        string       0          ['No', 'Yes', 'No phone service']
8   InternetService      string       0          ['DSL', 'Fiber optic', 'No']
9   OnlineSecurity       string       0          ['No', 'Yes', 'No internet service']
10  OnlineBackup         string       0          ['Yes', 'No', 'No internet service']
11  DeviceProtection     string       0          ['

##Entendiendo datos vacios

In [198]:
churn_nan = dataFrame[dataFrame['Churn'].isin(['', ' ', np.nan, None])].copy()
info(churn_nan)

#   Columna              Tipo         Indet.     Valores Únicos / Ejemplo
------------------------------------------------------------------------------------------
0   Churn                string       224        ['']
1   gender               string       0          ['Female', 'Male']
2   SeniorCitizen        Int64        0          [0, 1]
3   Partner              string       0          ['No', 'Yes']
4   Dependents           string       0          ['No', 'Yes']
5   tenure               Int64        0          64 valores diferentes
6   PhoneService         string       0          ['Yes', 'No']
7   MultipleLines        string       0          ['Yes', 'No', 'No phone service']
8   InternetService      string       0          ['Fiber optic', 'No', 'DSL']
9   OnlineSecurity       string       0          ['Yes', 'No internet service', 'No']
10  OnlineBackup         string       0          ['No', 'No internet service', 'Yes']
11  DeviceProtection     string       0          ['No', 'No inte

In [199]:
chargesTotal_nan = dataFrame[dataFrame['Charges.Total'].isin(['', ' ', np.nan, None])].copy()
info(chargesTotal_nan)

#   Columna              Tipo         Indet.     Valores Únicos / Ejemplo
------------------------------------------------------------------------------------------
0   Churn                string       0          ['No']
1   gender               string       0          ['Female', 'Male']
2   SeniorCitizen        Int64        0          [0]
3   Partner              string       0          ['Yes', 'No']
4   Dependents           string       0          ['Yes']
5   tenure               Int64        0          [0]
6   PhoneService         string       0          ['No', 'Yes']
7   MultipleLines        string       0          ['No phone service', 'No', 'Yes']
8   InternetService      string       0          ['DSL', 'No']
9   OnlineSecurity       string       0          ['Yes', 'No internet service', 'No']
10  OnlineBackup         string       0          ['Yes', 'No internet service', 'No']
11  DeviceProtection     string       0          ['Yes', 'No internet service', 'No']
12  TechSupport   

In [200]:
tenure_zero = dataFrame[dataFrame['tenure'] == 0].copy()
info(tenure_zero)

#   Columna              Tipo         Indet.     Valores Únicos / Ejemplo
------------------------------------------------------------------------------------------
0   Churn                string       0          ['No']
1   gender               string       0          ['Female', 'Male']
2   SeniorCitizen        Int64        0          [0]
3   Partner              string       0          ['Yes', 'No']
4   Dependents           string       0          ['Yes']
5   tenure               Int64        0          [0]
6   PhoneService         string       0          ['No', 'Yes']
7   MultipleLines        string       0          ['No phone service', 'No', 'Yes']
8   InternetService      string       0          ['DSL', 'No']
9   OnlineSecurity       string       0          ['Yes', 'No internet service', 'No']
10  OnlineBackup         string       0          ['Yes', 'No internet service', 'No']
11  DeviceProtection     string       0          ['Yes', 'No internet service', 'No']
12  TechSupport   

###
  1 - No se detecta correlación en los "churn" Indeterminados. Se procede a eliminar esas filas porque no aportan datos utiles para el análisis.

  2 - Se observa una correlacion entre "tenure" igual a 0 y "Charge.Total" vacio. Se procede a rellenarlo con 0.

  3- 'No internet service' y 'No phone service' presenta Multicolinealidad se modificará a No

  4- El formato Charges.Total debe ser Float

##Manejo de inconsistencias

In [201]:
#Trasnformando vacios a np.nan
dataFrame = dataFrame.replace(r'^\s*$', np.nan, regex=True)
#Trasformando 'No internet service' y 'No phone service' (Multicolinealidad) a No
dataFrame = dataFrame.replace('No internet service', 'No')
dataFrame = dataFrame.replace('No phone service', 'No')

#Eliminado filas donde Churn es vacio
dataFrame.dropna(subset=['Churn'], inplace=True)
dataFrame.reset_index(drop=True, inplace=True)

#Transformando 'Charges.Total' en float64
dataFrame['Charges.Total'] = pd.to_numeric(dataFrame['Charges.Total'], errors='coerce')
#Transformando Charges.Total vacios a 0
dataFrame['Charges.Total'] = dataFrame['Charges.Total'].fillna(0)

dataFrame['Charges.Total'] = dataFrame['Charges.Total'].astype('Float64')
info(dataFrame)

#   Columna              Tipo         Indet.     Valores Únicos / Ejemplo
------------------------------------------------------------------------------------------
0   Churn                string       0          ['No', 'Yes']
1   gender               string       0          ['Female', 'Male']
2   SeniorCitizen        Int64        0          [0, 1]
3   Partner              string       0          ['Yes', 'No']
4   Dependents           string       0          ['Yes', 'No']
5   tenure               Int64        0          73 valores diferentes
6   PhoneService         string       0          ['Yes', 'No']
7   MultipleLines        string       0          ['No', 'Yes']
8   InternetService      string       0          ['DSL', 'Fiber optic', 'No']
9   OnlineSecurity       string       0          ['No', 'Yes']
10  OnlineBackup         string       0          ['Yes', 'No']
11  DeviceProtection     string       0          ['No', 'Yes']
12  TechSupport          string       0          ['Yes', '

##Creación de nueva columna para obtener los cargos por dia


In [202]:
dataFrame['Charges.Daily'] = dataFrame['Charges.Monthly'] / 30

##Transformación de variables categóricas

In [203]:
# Definimos el diccionario de mapeo
mapping = {'Yes': 1, 'No': 0}

for col in dataFrame.columns:
    unique_values = set(dataFrame[col].dropna().unique())

    # Verificamos que sea una columna de Yes/No
    if unique_values.issubset({'Yes', 'No'}) and len(unique_values) > 0:
        # Usamos Int64 (mayúscula) para mantener la consistencia
        dataFrame[col] = dataFrame[col].map(mapping).astype('Int64')

info(dataFrame)

#   Columna              Tipo         Indet.     Valores Únicos / Ejemplo
------------------------------------------------------------------------------------------
0   Churn                Int64        0          [0, 1]
1   gender               string       0          ['Female', 'Male']
2   SeniorCitizen        Int64        0          [0, 1]
3   Partner              Int64        0          [1, 0]
4   Dependents           Int64        0          [1, 0]
5   tenure               Int64        0          73 valores diferentes
6   PhoneService         Int64        0          [1, 0]
7   MultipleLines        Int64        0          [0, 1]
8   InternetService      string       0          ['DSL', 'Fiber optic', 'No']
9   OnlineSecurity       Int64        0          [0, 1]
10  OnlineBackup         Int64        0          [1, 0]
11  DeviceProtection     Int64        0          [0, 1]
12  TechSupport          Int64        0          [1, 0]
13  StreamingTV          Int64        0          [1, 0]
14

#📊 Carga y análisis

##Análisis descriptivo de los datos


In [204]:
dataFrame.describe()

,Churn,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,PaperlessBilling,Charges.Monthly,Charges.Total,Charges.Daily
count,7043.0,7043.0,7043.0,7043.0,7043.0,7043.0,7043.0,7043.0,7043.0,7043.0,7043.0,7043.0,7043.0,7043.0,7043.0,7043.0,7043.0
mean,0.26537,0.162147,0.483033,0.299588,32.371149,0.903166,0.421837,0.286668,0.344881,0.343888,0.290217,0.384353,0.387903,0.592219,64.761692,2279.734304,2.158723
std,0.441561,0.368612,0.499748,0.45811,24.559481,0.295752,0.493888,0.452237,0.475363,0.475038,0.453895,0.486477,0.487307,0.491457,30.090047,2266.79447,1.003002
min,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,18.25,0.0,0.608333
25%,0.0,0.0,0.0,0.0,9.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,35.5,398.55,1.183333
50%,0.0,0.0,0.0,0.0,29.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,70.35,1394.55,2.345
75%,1.0,0.0,1.0,1.0,55.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,89.85,3786.6,2.995
max,1.0,1.0,1.0,1.0,72.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,118.75,8684.8,3.958333
